# Reading proprietary image formats (here, ND2) in python

Anyone who has used Fiji has probably come to know and love the Bioformats plugin, which allows Fiji to read in pretty much any of the proprietary image formats from leica, olympus, zeiss, nikon, etc...  

While we *can* directly use bioformats in python code using packages such as [python-bioformats](https://pythonhosted.org/python-bioformats) or [pims](http://soft-matter.github.io/pims/v0.5/bioformats.html) (shown below), because bioformats itself is written in Java, it means that we must have the java virtual machine installed and running in the background of our python process (that's what the aforementioned packages do).  While that's fine (it may be your only option!), I generally prefer to use "pure python" packages when available – meaning the code that reads the file header and loads the data from the file is written entirely in python (or sometimes C), and doesn't require loading bioformats in the background, which can be slower, and typically comes with an "unpythonic" user interface.

We saw an example of using `tifffile` in the last segment, but that's a pretty easy case.  Here we will look at various solutions for reading in a nikon nd2 file.

### using `nd2reader`

https://github.com/rbnvrw/nd2reader

In [21]:
import numpy as np
from nd2reader import ND2Reader as nd2r

filename = 'data/elements_ex.nd2'
images = nd2r(filename)
print(images)


<FramesSequenceND>
Axes: 5
Axis 'x' size: 256
Axis 'y' size: 256
Axis 'c' size: 2
Axis 't' size: 1
Axis 'z' size: 5
Pixel Datatype: <class 'numpy.float64'>


In [22]:
# includes metadata
images.metadata

{'height': 256,
 'width': 256,
 'date': datetime.datetime(2021, 6, 23, 13, 17, 17),
 'fields_of_view': [0],
 'frames': [0],
 'z_levels': range(0, 5),
 'z_coordinates': [2283.1, 2284.04, 2284.88, 2285.84, 2286.7400000000002],
 'total_images_per_channel': 5,
 'channels': ['Widefield Dual Green', 'Widefield Dual Red'],
 'pixel_microns': 0.325,
 'num_frames': 1,
 'experiment': {'description': 'ND Acquisition', 'loops': []},
 'events': []}

### using `pims_nd2`

https://github.com/soft-matter/pims_nd2

In [33]:
from pims import ND2Reader_SDK as pnd2


images = pnd2(filename)
print(images)

<FramesSequenceND>
Axes: 4
Axis 'x' size: 256
Axis 'y' size: 256
Axis 'c' size: 2
Axis 'z' size: 5
Pixel Datatype: <class 'numpy.uint16'>


In [34]:
images.metadata

{'width': 256,
 'width_bytes': 1024,
 'height': 256,
 'components': 2,
 'bitsize_memory': 16,
 'bitsize_significant': 16,
 'sequence_count': 5,
 'tile_width': 256,
 'tile_height': 256,
 'compression': None,
 'compression_quality': 4294967197,
 'plane_count': 2,
 'angle': -0.016189802964737033,
 'calibration_um': 0.325,
 'time_start_jdn': 2459389.2203407986,
 'time_start': datetime.datetime(2021, 6, 23, 13, 17, 17, 445000),
 'time_start_utc': datetime.datetime(2021, 6, 23, 17, 17, 17, 445000),
 'objective': 'Plan Apo λ 20x Ph2 DM',
 'magnification': -1.0,
 'NA': 0.75,
 'refractive_index1': 1.0,
 'refractive_index2': 1.0,
 'pinhole': 0.0,
 'zoom': 1.0,
 'projective_mag': -1.0,
 'image_type': 'normal',
 'z_home': 2,
 'frame_rate': None,
 'plane_0': {'components': 1,
  'rgb_value': (0.21176470588235294, 1.0, 0.0),
  'name': 'Widefield Dual Green',
  'oc': 'Widefield Dual Green',
  'emission_nm': 525.0},
 'plane_1': {'components': 1,
  'rgb_value': (0.996078431372549, 0.0, 0.0),
  'name': '

### using `pims-bioformats`

pims also comes with a module capable of interfacing with the java-based bioformats library

http://soft-matter.github.io/pims/v0.5/bioformats.html

... but we have to additionally install the [`Jpype`](https://github.com/jpype-project/jpype) library that provides access to java from within python.  (It's included in the `requirements.txt` file in this repo, so you may already have it).

In [35]:
from pims.bioformats import BioformatsReader

In [36]:
reader = BioformatsReader(filename)  # this will download the bioformats .jar if it hasn't already
reader

<FramesSequenceND>
Axes: 4
Axis 'x' size: 256
Axis 'y' size: 256
Axis 'c' size: 2
Axis 'z' size: 5
Pixel Datatype: <u2

In [37]:
# OME metadata is available, but this time you have to use the OME API:
# https://www-legacy.openmicroscopy.org/site/support/bio-formats5.1/developers/cpp/tutorial.html
meta = reader.metadata

image_count = meta.ImageCount()
print('Total number of images: {}'.format(image_count))

for i in range(image_count):
    print('Dimensions for image {}'.format(i))
    shape = (meta.PixelsSizeX(i), meta.PixelsSizeY(i), meta.PixelsSizeZ(i))
    dxyz = (meta.PixelsPhysicalSizeX(i),
            meta.PixelsPhysicalSizeY(i),
            meta.PixelsPhysicalSizeZ(i))
    print('\tShape: {} x {} x {}'.format(*shape))
    print('\tDxyz:  {:2.2f} x {:2.2f} x {:2.2f}'.format(*dxyz))

Total number of images: 1
Dimensions for image 0
	Shape: 256 x 256 x 5
	Dxyz:  0.33 x 0.33 x 0.90
